### Setup

In [136]:
import pandas as pd
import unicodedata
import json
import os
import re

In [137]:
bairros_fortaleza = [
    "aeroporto", "aerolandia", "alagadico_novo", "aldeota", "alto_da_balanca",
    "alvaro_weyne", "amadeu_furtado", "ancuri", "antonio_bezerra", "autran_nunes",
    "barra_do_ceara", "barroso", "bela_vista", "benfica", "bom_futuro",
    "bom_jardim", "bonsucesso", "cais_do_porto", "cajazeiras", "cambeba",
    "canindezinho", "carlito_pamplona", "cidade_2000", "cidade_dos_funcionarios", "coaçu",
    "coco", "conjunto_ceara_i", "conjunto_ceara_ii", "conjunto_esperanca", "conjunto_palmeiras",
    "couto_fernandes", "cristo_redentor", "curio", "damas", "democrito_rocha",
    "dende", "dias_macedo", "dionisio_torres", "dom_lustosa", "edson_queiroz",
    "ellery", "engenheiro_luciano_cavalcante", "farias_brito", "fatima", "floresta",
    "genibau", "granja_lisboa", "granja_portugal", "guajeru", "guararapes",
    "henrique_jorge", "itaoca", "itaperi", "jacarecanga", "jangurussu",
    "jardim_america", "jardim_cearense", "jardim_das_oliveiras", "jardim_guanabara", "jardim_iracema",
    "joao_xxiii", "joaquim_tavora", "joquei_clube", "jose_bonifacio", "jose_de_alencar",
    "lagoa_redonda", "manuel_dias_branco", "manoel_satiro", "maraponga", "messejana",
    "meireles", "mondubim", "monte_castelo", "montese", "moura_brasil",
    "mucuripe", "panamericano", "papicu", "parangaba", "parque_araxa",
    "parque_dois_irmaos", "parque_iracema", "parque_manibura", "parquelandia", "parreao",
    "passare", "paupina", "pedras", "pici", "pirambu",
    "planalto_ayrton_senna", "praia_de_iracema", "praia_do_futuro_i", "praia_do_futuro_ii", "jose_walter",
    "presidente_kennedy", "presidente_vargas", "quintino_cunha", "rodolfo_teofilo", "sabiaguaba",
    "salinas", "santa_rosa", "sao_bento", "sao_gerardo", "sao_joao_do_tauape",
    "sao_jose", "sapiranga", "serrinha", "siqueira", "varjota",
    "vicente_pinzon", "vila_ellery", "vila_peri", "vila_uniao", "vila_velha"
]

In [138]:
variacoes_bairros = {
   "aeroporto": ["aeroporto"],
    "aerolandia": ["aerolandia", "aerolândia", "alto da balança"],
    "alagadico_novo": ["alagadico novo", "alagadiço novo"],
    "aldeota": ["aldeota", "outeiro"],
    "alto_da_balanca": ["alto da balança"],
    "alvaro_weyne": ["alvaro weyne", "álvaro weyne", "floresta"],
    "amadeu_furtado": ["amadeu furtado", "coqueirinho"],
    "ancuri": ["ancuri", "aricuri"],
    "antonio_bezerra": ["antonio bezerra", "antônio bezerra", "barro vermelho"],
    "autran_nunes": ["autran nunes"],
    "barra_do_ceara": ["barra do ceara", "barra do ceará"],
    "barroso": ["barroso"],
    "bela_vista": ["bela vista"],
    "benfica": ["benfica"],
    "bom_futuro": ["bom futuro"],
    "bom_jardim": ["bom jardim"],
    "bonsucesso": ["bonsucesso"],
    "cais_do_porto": ["cais do porto"],
    "cajazeiras": ["cajazeiras"],
    "cambeba": ["cambeba", "cabeça chata"],
    "canindezinho": ["canindezinho"],
    "carlito_pamplona": ["carlito pamplona", "brasil oiticica"],
    "cidade_2000": ["cidade 2000"],
    "cidade_dos_funcionarios": ["cidade dos funcionarios", "cidade dos funcionários"],
    "coacu": ["coacu", "coaçu", "folha grande"],
    "coco": ["coco", "cocó"],
    "conjunto_ceara_i": ["conjunto ceara i", "conjunto ceará i"],
    "conjunto_ceara_ii": ["conjunto ceara ii", "conjunto ceará ii"],
    "conjunto_esperanca": ["conjunto esperanca", "conjunto esperança"],
    "conjunto_palmeiras": ["conjunto palmeiras"],
    "couto_fernandes": ["couto fernandes", "quilômetro 8", "km 8"],
    "cristo_redentor": ["cristo redentor"],
    "curio": ["curio", "curió", "amigo do homem"],
    "damas": ["damas"],
    "democrito_rocha": ["democrito rocha", "demócrito rocha", "maraponga"],
    "dende": ["dende", "dendê"],
    "dias_macedo": ["dias macedo"],
    "dionisio_torres": ["dionisio torres", "dionísio torres", "estância"],
    "dom_lustosa": ["dom lustosa"],
    "edson_queiroz": ["edson queiroz", "água fria"],
    "ellery": ["ellery", "vila ellery"],
    "engenheiro_luciano_cavalcante": ["engenheiro luciano cavalcante"],
    "farias_brito": ["farias brito", "farias_brito"],
    "fatima": ["fatima", "fátima"],
    "floresta": ["floresta", "bom jardim"],
    "genibau": ["genibau", "genibaú"],
    "granja_lisboa": ["granja lisboa", "granja lisbôa"],
    "granja_portugal": ["granja portugal"],
    "guajeru": ["guajeru", "guaijirú"],
    "guararapes": ["guararapes"],
    "henrique_jorge": ["henrique jorge", "casa popular"],
    "itaoca": ["itaoca"],
    "itaperi": ["itaperi"],
    "jacarecanga": ["jacarecanga"],
    "jangurussu": ["jangurussu"],
    "jardim_america": ["jardim america", "jardim américa"],
    "jardim_cearense": ["jardim cearense"],
    "jardim_das_oliveiras": ["jardim das oliveiras"],
    "jardim_guanabara": ["jardim guanabara"],
    "jardim_iracema": ["jardim iracema"],
    "joao_xxiii": ["joao xxiii", "joão xxiii"],
    "joaquim_tavora": ["joaquim tavora", "joaquim távora"],
    "joquei_clube": ["joquei clube", "jóquei clube"],
    "jose_bonifacio": ["jose bonifacio", "josé bonifácio"],
    "jose_de_alencar": ["jose de alencar", "josé de alencar"],
    "lagoa_redonda": ["lagoa redonda"],
    "manuel_dias_branco": ["manuel dias branco", "manuel d. branco"],
    "manoel_satiro": ["manoel satiro", "manoel sátiro"],
    "maraponga": ["maraponga"],
    "messejana": ["messejana"],
    "meireles": ["meireles"],
    "mondubim": ["mondubim"],
    "monte_castelo": ["monte castelo"],
    "montese": ["montese", "pirocaia"],
    "moura_brasil": ["moura brasil"],
    "mucuripe": ["mucuripe"],
    "panamericano": ["panamericano"],
    "papicu": ["papicu"],
    "parangaba": ["parangaba"],
    "parque_araxa": ["parque araxa", "parque araxá"],
    "parque_dois_irmaos": ["parque dois irmaos", "parque dois irmãos"],
    "parque_iracema": ["parque iracema"],
    "parque_manibura": ["parque manibura"],
    "parquelandia": ["parquelandia", "parquelândia", "campo do pio"],
    "parreao": ["parreao", "parreão"],
    "passare": ["passare", "passaré"],
    "paupina": ["paupina"],
    "pedras": ["pedras"],
    "pici": ["pici"],
    "pirambu": ["pirambu"],
    "planalto_ayrton_senna": ["planalto ayrton senna", "pantanal"],
    "praia_de_iracema": ["praia de iracema"],
    "praia_do_futuro_i": ["praia do futuro i"],
    "praia_do_futuro_ii": ["praia do futuro ii"],
    "prefeito_jose_walter": ["jose_walter", "josé walter", "jose walter", "prefeito jose walter", "prefeito josé walter"],
    "presidente_kennedy": ["presidente kennedy"],
    "presidente_vargas": ["presidente vargas"],
    "quintino_cunha": ["quintino cunha"],
    "rodolfo_teofilo": ["rodolfo teofilo", "rodolfo teófilo", "porangabussu"],
    "sabiaguaba": ["sabiaguaba"],
    "salinas": ["salinas"],
    "santa_rosa": ["santa rosa"],
    "sao_bento": ["sao bento", "são bento"],
    "sao_gerardo": ["sao gerardo", "são gerardo", "alagadiço"],
    "sao_joao_do_tauape": ["sao joao do tauape", "são joão do tauape"],
    "sao_jose": ["sao jose", "são josé"],
    "sapiranga": ["sapiranga"],
    "serrinha": ["serrinha"],
    "siqueira": ["siqueira"],
    "varjota": ["varjota"],
    "vicente_pinzon": ["vicente pinzon"],
    "vila_ellery": ["vila ellery"],
    "vila_peri": ["vila peri"],
    "vila_uniao": ["vila uniao", "vila união"],
    "vila_velha": ["vila velha"]
}

In [139]:
cidades_ceara = [
    "Acarape", "Acaraú", "Alcântaras", "Altaneira", "Alto Santo", "Amontada",
    "Aquiraz", "Aracati", "Ararendá", "Aratuba", "Baturité", "Beberibe", 
    "Bela Cruz", "Boa Viagem", "Brejo Santo", "Camocim", "Campos Sales", 
    "Canindé", "Capistrano", "Caridade", "Caririacu", "Cascavel", "Catarina", 
    "Cedro", "Choró", "Chorozinho", "Coreau", "Crateús", "Crato", "Croatá", 
    "Cruz", "Deputado Irapuan Pinheiro", "Ereré", "Eusébio", "Farias Brito", 
    "Forquilha", "Frecheirinha", "Guaiúba", "Guaraciaba do Norte",
    "Hidrolândia", "Horizonte", "Ibaretama", "Ibiapina", "Icapuí", "Icó", "Iracema",
    "Itaiçaba", "Itapagé", "Itapipoca", "Itatira", "Jaguaretama", "Jaguaruana", 
    "Jardim", "Jati", "Juazeiro do Norte", "Jucás", "Lavras da Mangabeira", "Limoeiro do Norte",
    "Madalena", "Maracanaú", "Marco", "Martinópole", "Massapê", "Mauriti", "Meruoca", 
    "Milagres", "Moraújo", "Morrinhos", "Mucambo", "Mulungu", "Nova Olinda", "Novo Oriente",
    "Pacajus", "Pacatuba", "Pacoti", "Palhano", "Paracuru", "Paraipaba", "Paramoti", "Pedra Branca",
    "Penaforte", "Pereiro", "Pindoretama", "Piquet Carneiro", "Poranga", "Porteiras", "Potengi", 
    "Quiterianópolis", "Quixadá", "Quixeramobim", "Quixeré", "Redenção", "Reriutaba", "Russas", 
    "Salitre", "Santa Quitéria", "São Benedito", "São Gonçalo do Amarante", "São João do Jaguaribe",
    "São Luís do Curu", "Senador Pompeu", "Senador Sá", "Sobral", "Solonópole", "Tamboril", 
    "Tarrafas", "Tauá", "Tejuçuoca", "Tianguá", "Trairi", "Tururu", "Ubajara", "Uruoca", 
    "Varjota", "Viçosa do Ceará"
]

In [140]:
def normalizar_texto(texto):
    """ 
    Removendo acentos e convertendo para minúsculas
    E tratando variações dos bairros para um formato consistente
    """
    texto_normalizado = ''.join(
        c for c in unicodedata.normalize('NFD', texto)
        if unicodedata.category(c) != 'Mn'
    ).lower()
    for bairro, variações in variacoes_bairros.items():
        for variacao in variações:
            texto_normalizado = texto_normalizado.replace(variacao, bairro)
    
    return texto_normalizado

In [141]:
def verifica_cidade_na_mesma_frase(message, bairro, cidades_ceara):
    # Verifica se há menção a cidades fora de Fortaleza na mesma frase que o bairro
    # A regex irá procurar por qualquer nome de cidade fora de Fortaleza
    for cidade in cidades_ceara:
        if cidade.lower() != 'fortaleza' and cidade.lower() in message.lower():
            if bairro.lower() in message.lower() and cidade.lower() not in bairro.lower():
                return True 
    return False


In [142]:
bairros_normalizados = [normalizar_texto(bairro) for bairro in bairros_fortaleza]
pd.set_option('display.max_colwidth', None)

In [143]:
local_folder = "./downloads/Portalnoticiasceara/past_dates/" 
dados = []

In [144]:
from datetime import datetime, timedelta

def verificar_ultimas_24_horas(data_msg):
    try:
        data_msg = datetime.strptime(data_msg, "%Y-%m-%d %H:%M:%S")
        agora = datetime.now()
        diff = agora - data_msg
        return diff <= timedelta(hours=24)
    except Exception as e:
        print(f"Erro ao verificar a data: {e}")
        return False
    

def verificar_ultimos_7_dias(data_msg):
    try:
        data_msg = datetime.strptime(data_msg, "%Y-%m-%d %H:%M:%S")
        agora = datetime.now()
        diff = agora - data_msg
        return diff <= timedelta(days=7)
    except Exception as e:
        print(f"Erro ao verificar a data: {e}")
        return False
    

def verificar_ultimo_mes(data_msg):
    try:
        data_msg = datetime.strptime(data_msg, "%Y-%m-%d %H:%M:%S")
        agora = datetime.now()
        diff = agora - data_msg
        return diff <= timedelta(days=30)
    except Exception as e:
        print(f"Erro ao verificar a data: {e}")
        return False


### Funções

In [145]:
def create_json_file(nome_pasta, funcao_offset):
    dados = []
    faccoes_list = ["CV", "PCC", "GDE", "TDN", "MASSA"]
    faccoes_regex = r'\s?\(({})\)\s?'.format('|'.join(faccoes_list))
    ids_processados = set()

    for root, dirs, files in os.walk(local_folder):
        for file in files:
            if file.endswith('.json'):
                file_path = os.path.join(root, file)
                print(f"Lendo o arquivo: {file_path}")
                try:
                    with open(file_path, 'r', encoding='utf-8') as json_file:
                        json_data = json.load(json_file)

                    if isinstance(json_data, list):
                        for message_data in json_data:
                            message = message_data.get('message', '')
                            date = message_data.get('date', None)
                            media_path = message_data.get('media_paths', [])
                            message_id = message_data.get('id', '')

                            if not message or not date or not message_id:
                                continue
                            if message_id in ids_processados:
                                continue
                            if not funcao_offset(date):
                                continue

                            message_normalizada = normalizar_texto(message)

                            faccao_detectada = None
                            match = re.search(faccoes_regex, message)
                            if match:
                                faccao_detectada = match.group(1)

                            bairros_mencionados = [
                                bairro_original for bairro_original, bairro_normalizado in zip(bairros_fortaleza, bairros_normalizados)
                                if bairro_normalizado in message_normalizada
                            ]

                            if bairros_mencionados:
                                primeiro_bairro = bairros_mencionados[0]

                                # Converte a data para o formato correto (sem fuso horário)
                                data_naive = datetime.strptime(date, "%Y-%m-%d %H:%M:%S").strftime("%Y-%m-%d %H:%M:%S")

                                dados.append({
                                    "id": message_id,
                                    "mensagem": message,
                                    "data": data_naive,
                                    "media_path": media_path if media_path else [],  # Verifica se media_path está vazio
                                    "faccao": faccao_detectada,
                                    "bairro": primeiro_bairro
                                })
                                ids_processados.add(message_id)

                except Exception as e:
                    print(f"Erro ao processar {file_path}: {e}")

    output_folder = f"../data/Portalnoticiasceara/{nome_pasta}"
    os.makedirs(output_folder, exist_ok=True)

    output_file = os.path.join(output_folder, 'metadata.json')
    with open(output_file, 'w', encoding='utf-8') as json_file:
        json.dump(dados, json_file, ensure_ascii=False, indent=4)

    print(f"Dados processados e salvos em {output_file}")


In [146]:
def create_json_agg_bairro():
    agrupado_por_bairro = {}
    faccoes_list = ["CV", "PCC", "GDE", "TDN", "MASSA"]
    faccoes_regex = r'\s?\(({})\)\s?'.format('|'.join(faccoes_list)) 

    for root, dirs, files in os.walk(local_folder):
        for file in files:
            if file.endswith('.json'):
                file_path = os.path.join(root, file)
                try:
                    with open(file_path, 'r', encoding='utf-8') as json_file:
                        json_data = json.load(json_file)

                    if isinstance(json_data, list):
                        for message_data in json_data:
                            message_id = message_data.get('id')
                            message = message_data.get('message', '')
                            date = message_data.get('date', None)
                            media_path = message_data.get('media_paths', None)

                            if message:
                                faccao_detectada = None
                                match = re.search(faccoes_regex, message)
                                if match:
                                    faccao_detectada = match.group(1)

                                message_normalizada = normalizar_texto(message)
                                bairros_mencionados = {
                                    bairro: message_normalizada.find(bairro) 
                                    for bairro in variacoes_bairros 
                                    if bairro in message_normalizada
                                }

                                bairros_ordenados = sorted(bairros_mencionados.items(), key=lambda item: item[1])

                                if bairros_ordenados:
                                    bairro = bairros_ordenados[0][0]

                                    if not verifica_cidade_na_mesma_frase(message, bairro, cidades_ceara):
                                        if bairro not in agrupado_por_bairro:
                                            agrupado_por_bairro[bairro] = []

                                        mensagem_existente = any(
                                            mensagem['id'] == message_id and mensagem['mensagem'] == message
                                            for mensagem in agrupado_por_bairro[bairro]
                                        )

                                        if not mensagem_existente:
                                            # Aqui ajustamos a conversão da data para evitar erro de fuso horário
                                            data_naive = datetime.strptime(date, "%Y-%m-%d %H:%M:%S").strftime("%Y-%m-%d %H:%M:%S") if date else None
                                            agrupado_por_bairro[bairro].append({
                                                "id": message_id,
                                                "mensagem": message,
                                                "data": data_naive,
                                                "media_path": media_path,
                                                "faccao": faccao_detectada,
                                                "bairro": bairro
                                            })
                    elif isinstance(json_data, dict):
                        for message_id, message_data in json_data.items():
                            message = message_data.get('message', '')
                            date = message_data.get('date', None)
                            media_path = message_data.get('media_path', None)

                            if message:
                                faccao_detectada = None
                                match = re.search(faccoes_regex, message)
                                if match:
                                    faccao_detectada = match.group(1)

                                message_normalizada = normalizar_texto(message)
                                bairros_mencionados = {
                                    bairro: message_normalizada.find(bairro) 
                                    for bairro in variacoes_bairros 
                                    if bairro in message_normalizada
                                }

                                bairros_ordenados = sorted(bairros_mencionados.items(), key=lambda item: item[1])

                                if bairros_ordenados:
                                    bairro = bairros_ordenados[0][0]

                                    if not verifica_cidade_na_mesma_frase(message, bairro, cidades_ceara):
                                        if bairro not in agrupado_por_bairro:
                                            agrupado_por_bairro[bairro] = []

                                        mensagem_existente = any(
                                            mensagem['id'] == message_id and mensagem['mensagem'] == message
                                            for mensagem in agrupado_por_bairro[bairro]
                                        )

                                        if not mensagem_existente:
                                            # Aqui ajustamos a conversão da data para evitar erro de fuso horário
                                            data_naive = datetime.strptime(date, "%Y-%m-%d %H:%M:%S").strftime("%Y-%m-%d %H:%M:%S") if date else None
                                            agrupado_por_bairro[bairro].append({
                                                "id": message_id,
                                                "mensagem": message,
                                                "data": data_naive,
                                                "media_path": media_path,
                                                "faccao": faccao_detectada,
                                                "bairro": bairro
                                            })
                except Exception as e:
                    print(f"Erro ao processar {file_path}: {e}")

    output_folder = "./data/Portalnoticiasceara/uf/ceara/fortaleza/bairros"
    os.makedirs(output_folder, exist_ok=True)

    for bairro, mensagens in agrupado_por_bairro.items():
        output_file = os.path.join(output_folder, f"{bairro}.json")
        with open(output_file, 'w', encoding='utf-8') as json_file:
            json.dump(mensagens, json_file, ensure_ascii=False, indent=4)
        print(f"Arquivo criado para {bairro}: {output_file}")

### Run :)

In [147]:
create_json_file('noticias_recentes_ultimo_mes', verificar_ultimo_mes)

Dados processados e salvos em ../data/Portalnoticiasceara/noticias_recentes_ultimo_mes/metadata.json


In [148]:
create_json_file('noticias_recentes_ultimos_7_dias', verificar_ultimos_7_dias)

Dados processados e salvos em ../data/Portalnoticiasceara/noticias_recentes_ultimos_7_dias/metadata.json


In [149]:
create_json_file('noticias_recentes_ultimas_24_horas', verificar_ultimas_24_horas)

Dados processados e salvos em ../data/Portalnoticiasceara/noticias_recentes_ultimas_24_horas/metadata.json


In [150]:
create_json_agg_bairro()